# Code for generating and running PH of noisy networks - dev

In [ ]:
# Need to load in all the network generation functions here?


In [ ]:
# Load packages

using Pkg
using Statistics
using Plots
using LinearAlgebra
using Distances
using Eirene
using StatsBase
using Random
using Distributions
using JLD

include("graph_models.jl")
include("helper_functions.jl")


## Choose weighed network and generate replicates

--------------------------------
## CHANGE PARAMETERS HERE
---------------------------------

In [ ]:
## Pick from one of the graph models and generate out to THRESH edge density

nReps = 50
rho = 0.5      # Threshold at edge density = rho
nNodes = 70
save_figures = 1  # Boolean to save figure pdfs
save_data = 1    # Boolean to save data  

# for geometricConf
p = 0.01
scale_factor = 100

# for RG
dims = 3

# for discreteUniformConf
a = 0
b = 1000

# for cosineGeometric
dims = 3


# Graph model name one of "geometricConf", "IID" , "RG", "discreteUniformConf", "cosineGeometric", "RL", "assoc"
graph_model_name = "IID"

In [ ]:


weighted_graph_array = zeros(nNodes,nNodes,nReps)
weighted_graph_array_ord = zeros(nNodes,nNodes,nReps)
betti_file_name = []
parameters = []
for rep in 1:nReps
    
    
    if graph_model_name == "geometricConf"
        G_i = make_dev_Geometric_configuration_model(nNodes,p,scale_factor)
        parameters = [nReps, rho, nNodes, p, scale_factor]
        betti_file_name = "$(graph_model_name)"
        
        for parameter in parameters
            betti_file_name = "$(betti_file_name)_$(replace(string(parameter),"." => ""))"
        end
        
        
    elseif graph_model_name == "IID"
        G_i = make_iid_weighted_graph(nNodes)
        parameters = [nReps, rho, nNodes]
        betti_file_name = "$(graph_model_name)"
        
        for parameter in parameters
            betti_file_name = "$(betti_file_name)_$(replace(string(parameter),"." => ""))"
        end
        
        
    elseif graph_model_name == "RG"
        G_i = make_random_geometric(nNodes,dims)
        parameters = [nReps, rho, nNodes, dims]
        betti_file_name = "$(graph_model_name)"
        
        for parameter in parameters
            betti_file_name = "$(betti_file_name)_$(replace(string(parameter),"." => ""))"
        end
       
        
    elseif graph_model_name == "discreteUniformConf"
        G_i = make_dev_DiscreteUniform_configuration_model(nNodes,a,b)
        parameters = [nReps, rho, nNodes, a, b]
        betti_file_name = "$(graph_model_name)"
        
        for parameter in parameters
            betti_file_name = "$(betti_file_name)_$(replace(string(parameter),"." => ""))"
        end
        
        
    elseif graph_model_name == "cosineGeometric"
        G_i = make_cosine_geometric(nNodes,dims)
        parameters = [nReps, rho, nNodes, dims]
        betti_file_name = "$(graph_model_name)"
        
        for parameter in parameters
            betti_file_name = "$(betti_file_name)_$(replace(string(parameter),"." => ""))"
        end
        
        
    elseif graph_model_name == "RL"
        G_i = make_ring_lattice_wei(nNodes)
        parameters = [nReps, rho, nNodes]
        betti_file_name = "$(graph_model_name)"
        
        for parameter in parameters
            betti_file_name = "$(betti_file_name)_$(replace(string(parameter),"." => ""))"
        end
        
        
    elseif graph_model_name == "assoc"
        println("load data you fool")
    end
    
        

    
    # Threshold at rho
    G_i_thresh = threshold_graph(G_i,rho,nNodes)
    
    weighted_graph_array[:,:,rep] = G_i_thresh
    
    
    edge_list_ranks = denserank([G_i...], rev = true)   # so highest edge weight gets assigned 1
    G_i_ord = reshape(edge_list_ranks,(nNodes,nNodes))
    G_i_ord[diagind(G_i_ord)] .= 0
    weighted_graph_array_ord[:,:,rep] = G_i_ord
    
    
 
    
end
println("Naming files $(betti_file_name)")

# We can add noise to the entire array at the same time

weighted_graph_array_iidNoise = deepcopy(weighted_graph_array)
weighted_graph_array_iidNoise_ord = zeros(nNodes,nNodes,nReps)
for rep in 1:nReps
    
    G_i = weighted_graph_array_iidNoise[:,:,rep]
    G_i[G_i.>0] .= G_i[G_i .>0 ] .+1
    
    # Now the real values are all >1, so we can add noise which will be < 1
    
    G_rand = make_iid_weighted_graph(nNodes)
    G_i_rand = deepcopy(G_i)
    G_i_rand[G_i_rand .== 0] .= G_rand[G_i_rand .== 0]
    
    # So anything <1 in G_i_rand will be from random noise, and any entry >1 will be real matrix
    
    weighted_graph_array_iidNoise[:,:,rep] = G_i_rand
    
    # Now create the ranked matrix for running eirene -- it will save us many headaches later.
    edge_list_ranks = denserank([G_i_rand...], rev = true)   # so highest edge weight gets assigned 1
    G_i_rand_ord = reshape(edge_list_ranks,(nNodes,nNodes))
    G_i_rand_ord[diagind(G_i_rand_ord)] .= 0
    weighted_graph_array_iidNoise_ord[:,:,rep] = G_i_rand_ord
    
    
end



    
# Plot an example weighted graph and after we add noise

gr()

test_mat = 2
p0a = heatmap(weighted_graph_array[:,:,test_mat], aspect_ratio = :equal)
p0b = heatmap(weighted_graph_array_iidNoise[:,:,test_mat], aspect_ratio = :equal)
p0c = heatmap(weighted_graph_array_iidNoise_ord[:,:,test_mat], aspect_ratio = :equal)

plot(p0a,p0b,p0c,layout = (1,3))


if save_figures == 1
    savefig("../figures/$(betti_file_name).pdf")
end

if save_data == 1
    save("../processed_data/$(betti_file_name)_graphs.jld",
     "weighted_graph_array", weighted_graph_array,
     "weighted_graph_array_iidNoise", weighted_graph_array_iidNoise,
     "parameters", parameters)
end



## Running Eirene

In [ ]:
# Loop through replicates and run eirene

maxDim = 5
nEdges = binomial(nNodes,2)
barcode_array = Array{Array{Float64}}(undef,nReps,maxDim)
betticurve_array = zeros(nEdges,maxDim,nReps)
beginTime = time()



for rep = 1:nReps
    
    G_i = weighted_graph_array_iidNoise_ord[:,:,rep]
    
    C = Eirene.eirene(G_i,model = "vr", maxdim = maxDim, record = "none")
    println("finished running eirene")
    for k in collect(1:maxDim)
        
        barcode_array[rep,k] = barcode(C,dim=k)
        #betti_curve_i = betticurveFromBarcode(barcode_array[rep,k],nEdges)
        #betticurve_array[:,k,rep] = betti_curve_i[1,:]
        
    end 
   
    #if rep%10 == 0
        printstyled("\nRun $(rep) finished.\n",color = :cyan)
    #end

    C=0
    endTime = time() - beginTime
    printstyled("Finished computing persistent homology, elapsed time $(endTime) seconds", color = :cyan)

end



endTime = time() - beginTime
printstyled("Finished computing persistent homology, elapsed time $(endTime) seconds", color = :cyan)

if save_data == 1
    save("../processed_data/$(betti_file_name)_eirene_output.jld",
        "barcode_array", barcode_array,
        "betticurve_array", betticurve_array)
end


printstyled("\nDone :)", color = :blue)


In [ ]:
# Calculate betti curves from barcodes

beginTime = time()
for rep = 1:nReps
    
    for k in collect(1:maxDim)
        
        betti_curve_i = betticurveFromBarcode(barcode_array[rep,k],nEdges)
        betticurve_array[:,k,rep] = betti_curve_i[1,:]
        
    end 
   
    #if rep%10 == 0
        printstyled("\nRun $(rep) finished.\n",color = :cyan)
    #end


    endTime = time() - beginTime
    printstyled("Finished computing betti curves, elapsed time $(endTime) seconds", color = :cyan)

end

In [ ]:
## Plot average betti curves
### UPDATE FOR DIM 5

if save_figures == 1
    betticurve_avg = dropdims(mean(betticurve_array,dims=3), dims = 3)
    betticurve_std = dropdims(std(betticurve_array,dims=3), dims = 3)

    x_axis = collect(1:nEdges) ./nEdges
    p1a = plot(x_axis,betticurve_avg[:,1],linewidth = 1.5, ribbon = betticurve_std[:,1], label = "Betti_1")
    for d = 2:maxDim
        plot!(x_axis,betticurve_avg[:,d],linewidth = 1.5, ribbon = betticurve_std[:,d], label = "Betti_$(d)")
    end

    title!("$(betti_file_name)")
    ylabel!("Betti_k")
    xlabel!("Edge Density")

    plot(p1a,layout = (1,1))


    savefig("../figures/$(betti_file_name)_bettis.pdf")
end
